In [10]:
import math
import numpy as np
from collections import namedtuple
from tqdm.notebook import tqdm
import sys
import os
import matplotlib.pyplot as plt
from itertools import product

plt.rcParams['figure.figsize'] = [12, 8]

Customer = namedtuple("Customer", ['index', 'demand', 'x', 'y'])

def length(customer1, customer2):
    return math.sqrt((customer1.x - customer2.x)**2 + (customer1.y - customer2.y)**2)

def plot_data(customers,*args):
    for i in customers:
        if i.index!=0:
            plt.plot(i.x, i.y,'k.')
            #plt.annotate(i.index,(i.x, i.y))
        else:
            plt.plot(i.x, i.y,'ro')
            plt.annotate('Dep.',(i.x, i.y))
    if args==None:
        return
    else:
        sol=args[0]
        cmap = plt.cm.rainbow
        c_idx=0
        for tour in sol:
            for i in range(len(tour)):
                plt.plot((customers[tour[i-1]].x, customers[tour[i]].x),
                 (customers[tour[i-1]].y, customers[tour[i]].y),'-',
                         color=cmap(c_idx/len(sol)))     
            c_idx+=1

            
def fits_capacity(v_tour,v_cap,customers):
    tot_demand=0
    for c in v_tour:
        tot_demand+=customers[c].demand
    if tot_demand<v_cap:
        return True
    else:
        return False

def tour_cost(v_tour,customers):
    cost=0
    for i in range(len(v_tour)):
        cost+=length(customers[v_tour[i-1]],customers[v_tour[i]])
    return cost

def tour_demand(v_tour,customers):
    dem=0
    for i in v_tour:
        dem+=customers[i].demand
    return dem

def place_customer(in_tour, customer, customers):
    best_tour = in_tour[:]
    best_tour.insert(1,customer)
    best_cost = tour_cost(best_tour, customers)

    for i in range(1,len(best_tour)-1):
        new_tour = in_tour[:]
        new_tour.insert(i, customer)
        new_cost = tour_cost(new_tour, customers)
        if new_cost<best_cost:
            best_cost = new_cost
            best_tour = new_tour
    return best_tour

def objec(tours,customers):
    obj=0
    for t in tours:
        obj+=tour_cost(t,customers)
    return obj

def idx_two_opt(sol, idx1, idx2):
    # makes the reversal 
    if idx1<idx2:
        sol1=sol[:idx1]
        sol2=list(reversed(sol[idx1:idx2+1]))
        sol3=sol[idx2+1:]
    else:
        sol1=sol[:idx2]
        sol2=list(reversed(sol[idx2:idx1+1]))
        sol3=sol[idx1+1:]
    new_sol=sol1+sol2+sol3
    return new_sol

def two_opt_search(tour,customers):
    nodeCount=len(tour)
    best_tour=tour
    best_obj=tour_cost(tour,customers)
    for i in range(1,nodeCount-1):
            for j in range(1,nodeCount-1):
                p_tour=idx_two_opt(best_tour,i,j)
                p_obj=tour_cost(p_tour,customers)
                if p_obj<best_obj:
                    best_tour=p_tour
                    best_obj=p_obj
    return best_tour

def four_swap(vehicle_tours,c1,c2,c3,c4,customers,v_cap):
    local_cpy_tours = vehicle_tours[:]
    # calculate initial objective
    start_obj = objec(vehicle_tours,customers)
    candidates = [c1,c2,c3,c4]
    c_tours = []
    for c in candidates:
        for i in range(len(vehicle_tours)):
            if c in vehicle_tours[i]:
                c_tours.append((c,i))
                break
    # instantiates the set of all currently used tours amongst sourced clients 
    used_tours = {}
    for c in c_tours:
        used_tours.add(c[1])
    # remove candidates from tours trying to realocate
    for c in c_tours:
        local_cpy_tours[c[1]].remove(c[0])
    # try to permute candidates between tours keeping the best objective
    best_obj = start_obj
    best_tours = vehicle_tours[:]
    for perms in product(used_tours,repeat=4):
        # checks if current perm fits capacity
        if (fits_capacity(local_cpy_tours[perms[0]]+candidates[0],v_cap,customers) and
            fits_capacity(local_cpy_tours[perms[1]]+candidates[1],v_cap,customers) and
            fits_capacity(local_cpy_tours[perms[2]]+candidates[2],v_cap,customers) and
            fits_capacity(local_cpy_tours[perms[3]]+candidates[3],v_cap,customers) ):
            # if it fits, set up candidate tours
            cand_tours = local_cpy_tours[:]
            for i in range(4):
                cand_tours[perms[i]] = place_customer(cand_tours[perms[i]],
                                                      candidates[i], customers)
            # calculate candidate objective
            cand_obj = objec(cand_tours,customers)
            if cand_obj < best_obj:
                best_obj = cand_obj
                best_tours = cand_tours[:]
    return best_tours

In [14]:

def vr_solver(input_data):
    # Currently returns infeasible solutions for some problems!

    # parse the input
    lines = input_data.split('\n')

    parts = lines[0].split()
    customer_count = int(parts[0])
    vehicle_count = int(parts[1])
    vehicle_capacity = int(parts[2])
    
    print('Customer Count:',customer_count)
    print('Vehicle Count',vehicle_count)
    print('Vehicle Cap.',vehicle_capacity)
    
    customers = []
    for i in range(1, customer_count+1):
        line = lines[i]
        parts = line.split()
        customers.append(Customer(i-1, int(parts[0]), float(parts[1]), float(parts[2])))

    #the depot is always the first customer in the input
    depot = customers[0] 

    # greedy solution
    # assign customers to vehicles sorted by a balance (alpha) of distance and demand

    
    obj_hist = []
    
    best_obj = float('Inf')
    best_tours = []
    
    alpha_n=50
    # Iterate in a range alpha taking the best!
    for alpha in tqdm(np.linspace(0,1,alpha_n)):
        vehicle_tours = []
        remaining_customers = set(customers)
        remaining_customers.remove(depot)
        for v in range(0, vehicle_count):
            # print "Start Vehicle: ",v
            vehicle_tours.append([])
            capacity_remaining = vehicle_capacity
            current_c=0
            full=False
            while len(remaining_customers)>0 and not full:
                sorted_rem_cust=sorted(remaining_customers,
                                        key=lambda x: alpha*(length(customers[current_c],x)) + 
                                       (1-alpha)*(-x.demand), reverse = False)
                for c in sorted_rem_cust:
                    if c.demand<=capacity_remaining:
                        capacity_remaining-=c.demand
                        remaining_customers.remove(c)
                        current_c=c.index
                        vehicle_tours[v].append(c.index)
                        break
                #check if vehicle is effectively full!!
                full=True
                for c in remaining_customers:
                    if c.demand<=capacity_remaining:
                        full=False
                        break
        # checks that the number of customers served is correct
        valid = False
        if sum([len(v) for v in vehicle_tours]) == len(customers) - 1:
            valid=True
        # calculates objective for valid tour
        if valid == True:
            # the formulation pressuposes '0' at the start and end of tour
            # adding '0' to start and end
            for t in vehicle_tours:
                t.insert(0,0)
                t.append(0)
            for i in range(len(vehicle_tours)):
                vehicle_tours[i] = two_opt_search(vehicle_tours[i],customers)
            obj=objec(vehicle_tours,customers)
            if obj<best_obj:
                best_obj=obj
                best_tours = vehicle_tours
                obj_hist.append(obj)
            else:
                obj_hist.append(obj)
        else:
            obj_hist.append(None)
    vehicle_tours = best_tours

    print('Starting cost = {}'.format(objec(best_tours,customers)))
    
    # try swapping clients between tours!!!
    for t_origin in range(len(best_tours)):
        print('Tour ({}/{})'.format(t_origin+1,len(best_tours)))
        #print(best_tours[t_origin])
        # go through all clients in each tour and try transfering them to other tours
        for c in best_tours[t_origin]:
            # don't move origin
            if c!=0:
                for t_target in range(len(best_tours)):
                    #print('T_target',best_tours[t_target])
                    # checks if target tour is different from origin tour
                    if t_origin != t_target:
                        for c_t in best_tours[t_target]:
                            if c_t!=0:
                                #print('ct:',c_t)
                                # only try swithing if tour fits
                                new_origin_tour = [i for i in best_tours[t_origin] if i != c]
                                new_target_tour = [i for i in best_tours[t_target] if i != c_t]
                                # if swapping doesn't violate capacity, check for objective
                                if ( fits_capacity(new_target_tour+[c],vehicle_capacity,customers)
                                   and fits_capacity(new_origin_tour+[c_t],vehicle_capacity,customers) ):
                                    new_origin_tour = place_customer(best_tours[t_origin]+[c_t],
                                                                     c, customers)
                                    new_target_tour = place_customer(best_tours[t_target]+[c],
                                                                     c, customers)

                                    prev_tour_cost_sum = ( tour_cost(best_tours[t_origin],customers) + 
                                                          tour_cost(best_tours[t_target],customers) )

                                    new_tour_cost_sum = ( tour_cost(new_origin_tour, customers) + 
                                                         tour_cost(new_target_tour,customers) )


                                    # if the sum of both tour costs is decreased, then the switch is good!
                                    print('Origin:{}, Target:{}, Cust_o.:{}, Cust_t.:{}, Fits! --- Diff. Cost Sum={}'.format(t_origin,
                                                                                         t_target,c,c_t,new_tour_cost_sum-prev_tour_cost_sum))
                                    if new_tour_cost_sum < prev_tour_cost_sum:
                                        print('Prev. Cost Sum={}, New Cost Sum={}'.format(prev_tour_cost_sum,
                                                                                     new_tour_cost_sum))
                                        print('>>>Moving client {} from tour {} to tour {}'.format(c,
                                                                                               t_origin,
                                                                                               t_target))
                                        best_tours[t_target] = new_target_tour
                                        best_tours[t_origin] = new_origin_tour
                        
                    

    
    #plot cost curve
    plt.subplot(1,2,1)
    plt.plot(np.linspace(0,1,alpha_n),obj_hist,'.-')
    
    #prints the customers locations
    plt.subplot(1,2,2)
    plot_data(customers,vehicle_tours)
    
    # calculate the cost of the solution; for each vehicle the length of the route
    obj=objec(vehicle_tours,customers)
    print('Cost:',obj)

    # prepare the solution in the specified output format
    outputData = '%.2f' % obj + ' ' + str(0) + '\n'
    for v in range(0, vehicle_count):
        outputData += ' '.join([str(customers[c].index) for c in vehicle_tours[v]]) + '\n'

    return outputData


In [15]:
notebook_path = os.path.abspath("Notebook.ipynb")
data_path = os.path.join(os.path.dirname(notebook_path), "data\\vrp_484_19_1")

if len(data_path) > 1:
    file_location = data_path.strip()
    with open(file_location, 'r') as input_data_file:
        input_data = input_data_file.read()
        print(vr_solver(input_data))

Customer Count: 484
Vehicle Count 19
Vehicle Cap. 1000



Starting cost = 1276.772917614919
Tour (1/19)
Origin:0, Target:18, Cust_o.:9, Cust_t.:209, Fits! --- Diff. Cost Sum=67.88225099390857
Origin:0, Target:18, Cust_o.:9, Cust_t.:230, Fits! --- Diff. Cost Sum=70.71067811865473
Origin:0, Target:18, Cust_o.:9, Cust_t.:252, Fits! --- Diff. Cost Sum=70.77798348825877
Origin:0, Target:18, Cust_o.:9, Cust_t.:231, Fits! --- Diff. Cost Sum=67.95291753300992
Origin:0, Target:18, Cust_o.:9, Cust_t.:273, Fits! --- Diff. Cost Sum=70.97944405968994
Origin:0, Target:18, Cust_o.:9, Cust_t.:293, Fits! --- Diff. Cost Sum=71.31370849898477
Origin:0, Target:18, Cust_o.:9, Cust_t.:312, Fits! --- Diff. Cost Sum=71.77857433030866
Origin:0, Target:18, Cust_o.:9, Cust_t.:330, Fits! --- Diff. Cost Sum=72.37105858887972
Origin:0, Target:18, Cust_o.:9, Cust_t.:347, Fits! --- Diff. Cost Sum=73.08748934490674
Origin:0, Target:18, Cust_o.:9, Cust_t.:363, Fits! --- Diff. Cost Sum=73.92361186897884
Origin:0, Target:18, Cust_o.:9, Cust_t.:378, Fits! --- Diff. Cost Sum=74.

Origin:1, Target:18, Cust_o.:7, Cust_t.:312, Fits! --- Diff. Cost Sum=66.78942115166063
Origin:1, Target:18, Cust_o.:7, Cust_t.:330, Fits! --- Diff. Cost Sum=67.38190541023172
Origin:1, Target:18, Cust_o.:7, Cust_t.:347, Fits! --- Diff. Cost Sum=68.09833616625875
Origin:1, Target:18, Cust_o.:7, Cust_t.:363, Fits! --- Diff. Cost Sum=68.93445869033087
Origin:1, Target:18, Cust_o.:7, Cust_t.:378, Fits! --- Diff. Cost Sum=69.88554964861956
Origin:1, Target:18, Cust_o.:7, Cust_t.:392, Fits! --- Diff. Cost Sum=70.94653300595078
Origin:1, Target:18, Cust_o.:7, Cust_t.:405, Fits! --- Diff. Cost Sum=72.11209213393084
Origin:1, Target:18, Cust_o.:7, Cust_t.:417, Fits! --- Diff. Cost Sum=73.37677377729759
Origin:1, Target:18, Cust_o.:7, Cust_t.:428, Fits! --- Diff. Cost Sum=74.73508082628501
Origin:1, Target:18, Cust_o.:7, Cust_t.:438, Fits! --- Diff. Cost Sum=76.18155210662866
Origin:1, Target:18, Cust_o.:7, Cust_t.:447, Fits! --- Diff. Cost Sum=77.71082851063574
Origin:1, Target:18, Cust_o.:7, 

Origin:2, Target:18, Cust_o.:12, Cust_t.:312, Fits! --- Diff. Cost Sum=69.40913774132304
Origin:2, Target:18, Cust_o.:12, Cust_t.:330, Fits! --- Diff. Cost Sum=70.00162199989413
Origin:2, Target:18, Cust_o.:12, Cust_t.:347, Fits! --- Diff. Cost Sum=70.71805275592115
Origin:2, Target:18, Cust_o.:12, Cust_t.:363, Fits! --- Diff. Cost Sum=71.55417527999325
Origin:2, Target:18, Cust_o.:12, Cust_t.:378, Fits! --- Diff. Cost Sum=72.50526623828196
Origin:2, Target:18, Cust_o.:12, Cust_t.:392, Fits! --- Diff. Cost Sum=73.56624959561319
Origin:2, Target:18, Cust_o.:12, Cust_t.:405, Fits! --- Diff. Cost Sum=74.73180872359322
Origin:2, Target:18, Cust_o.:12, Cust_t.:417, Fits! --- Diff. Cost Sum=75.99649036695999
Origin:2, Target:18, Cust_o.:12, Cust_t.:428, Fits! --- Diff. Cost Sum=77.35479741594742
Origin:2, Target:18, Cust_o.:12, Cust_t.:438, Fits! --- Diff. Cost Sum=78.80126869629103
Origin:2, Target:18, Cust_o.:12, Cust_t.:447, Fits! --- Diff. Cost Sum=80.33054510029811
Origin:2, Target:18, 

Origin:2, Target:18, Cust_o.:15, Cust_t.:428, Fits! --- Diff. Cost Sum=80.07242929563884
Origin:2, Target:18, Cust_o.:15, Cust_t.:438, Fits! --- Diff. Cost Sum=81.51890057598249
Origin:2, Target:18, Cust_o.:15, Cust_t.:447, Fits! --- Diff. Cost Sum=83.04817697998956
Origin:2, Target:18, Cust_o.:15, Cust_t.:455, Fits! --- Diff. Cost Sum=84.65505415326922
Origin:2, Target:18, Cust_o.:15, Cust_t.:462, Fits! --- Diff. Cost Sum=86.33452260006845
Origin:2, Target:18, Cust_o.:15, Cust_t.:468, Fits! --- Diff. Cost Sum=88.08179648786177
Origin:2, Target:18, Cust_o.:15, Cust_t.:473, Fits! --- Diff. Cost Sum=89.89233265212235
Origin:2, Target:18, Cust_o.:15, Cust_t.:477, Fits! --- Diff. Cost Sum=91.76184136769373
Origin:2, Target:18, Cust_o.:15, Cust_t.:480, Fits! --- Diff. Cost Sum=93.68629040733006
Origin:2, Target:18, Cust_o.:15, Cust_t.:482, Fits! --- Diff. Cost Sum=95.66190378969057
Origin:2, Target:18, Cust_o.:15, Cust_t.:481, Fits! --- Diff. Cost Sum=93.68629040733006
Origin:2, Target:18, 

...

Origin:12, Target:18, Cust_o.:370, Cust_t.:230, Fits! --- Diff. Cost Sum=115.39696961966999
Origin:12, Target:18, Cust_o.:370, Cust_t.:252, Fits! --- Diff. Cost Sum=115.464274989274
Origin:12, Target:18, Cust_o.:370, Cust_t.:231, Fits! --- Diff. Cost Sum=112.63920903402519
Origin:12, Target:18, Cust_o.:370, Cust_t.:273, Fits! --- Diff. Cost Sum=115.66573556070514
Origin:12, Target:18, Cust_o.:370, Cust_t.:293, Fits! --- Diff. Cost Sum=115.99999999999997
Origin:12, Target:18, Cust_o.:370, Cust_t.:312, Fits! --- Diff. Cost Sum=116.46486583132386
Origin:12, Target:18, Cust_o.:370, Cust_t.:330, Fits! --- Diff. Cost Sum=117.05735008989498
Origin:12, Target:18, Cust_o.:370, Cust_t.:347, Fits! --- Diff. Cost Sum=117.773780845922
Origin:12, Target:18, Cust_o.:370, Cust_t.:363, Fits! --- Diff. Cost Sum=118.6099033699941
Origin:12, Target:18, Cust_o.:370, Cust_t.:378, Fits! --- Diff. Cost Sum=119.56099432828282
Origin:12, Target:18, Cust_o.:370, Cust_t.:392, Fits! --- Diff. Cost Sum=120.62197768

Origin:12, Target:10, Cust_o.:339, Cust_t.:165, Fits! --- Diff. Cost Sum=90.62438736496381
Origin:12, Target:10, Cust_o.:339, Cust_t.:184, Fits! --- Diff. Cost Sum=93.33613980556913
Origin:12, Target:10, Cust_o.:339, Cust_t.:166, Fits! --- Diff. Cost Sum=92.65053270425432
Origin:12, Target:10, Cust_o.:339, Cust_t.:149, Fits! --- Diff. Cost Sum=92.15361924162113
Origin:12, Target:10, Cust_o.:339, Cust_t.:133, Fits! --- Diff. Cost Sum=91.85248572417953
Origin:12, Target:10, Cust_o.:339, Cust_t.:118, Fits! --- Diff. Cost Sum=91.75159898806777
Origin:12, Target:10, Cust_o.:339, Cust_t.:103, Fits! --- Diff. Cost Sum=88.92317186332161
Origin:12, Target:10, Cust_o.:339, Cust_t.:117, Fits! --- Diff. Cost Sum=89.03179707079266
Origin:12, Target:10, Cust_o.:339, Cust_t.:132, Fits! --- Diff. Cost Sum=89.3557697170977
Origin:12, Target:10, Cust_o.:339, Cust_t.:148, Fits! --- Diff. Cost Sum=89.88954376984753
Origin:12, Target:11, Cust_o.:339, Cust_t.:203, Fits! --- Diff. Cost Sum=94.20121132487839


Origin:12, Target:18, Cust_o.:322, Cust_t.:447, Fits! --- Diff. Cost Sum=121.74498643835565
Origin:12, Target:18, Cust_o.:322, Cust_t.:455, Fits! --- Diff. Cost Sum=123.35186361163531
Origin:12, Target:18, Cust_o.:322, Cust_t.:462, Fits! --- Diff. Cost Sum=125.03133205843457
Origin:12, Target:18, Cust_o.:322, Cust_t.:468, Fits! --- Diff. Cost Sum=126.77860594622788
Origin:12, Target:18, Cust_o.:322, Cust_t.:473, Fits! --- Diff. Cost Sum=128.58914211048844
Origin:12, Target:18, Cust_o.:322, Cust_t.:477, Fits! --- Diff. Cost Sum=130.45865082605982
Origin:12, Target:18, Cust_o.:322, Cust_t.:480, Fits! --- Diff. Cost Sum=132.38309986569615
Origin:12, Target:18, Cust_o.:322, Cust_t.:482, Fits! --- Diff. Cost Sum=134.35871324805666
Origin:12, Target:18, Cust_o.:322, Cust_t.:481, Fits! --- Diff. Cost Sum=132.38309986569615
Origin:12, Target:18, Cust_o.:322, Cust_t.:479, Fits! --- Diff. Cost Sum=130.45865082605982
Origin:12, Target:18, Cust_o.:322, Cust_t.:476, Fits! --- Diff. Cost Sum=128.589

Origin:12, Target:13, Cust_o.:323, Cust_t.:398, Fits! --- Diff. Cost Sum=110.99019513592779
Origin:12, Target:13, Cust_o.:323, Cust_t.:384, Fits! --- Diff. Cost Sum=109.02466765011545
Origin:12, Target:13, Cust_o.:323, Cust_t.:369, Fits! --- Diff. Cost Sum=107.13287052640058
Origin:12, Target:13, Cust_o.:323, Cust_t.:353, Fits! --- Diff. Cost Sum=105.32250596475231
Origin:12, Target:13, Cust_o.:323, Cust_t.:336, Fits! --- Diff. Cost Sum=103.60198088779143
Origin:12, Target:13, Cust_o.:323, Cust_t.:318, Fits! --- Diff. Cost Sum=101.98039027185567
Origin:12, Target:13, Cust_o.:323, Cust_t.:299, Fits! --- Diff. Cost Sum=100.4674626433397
Origin:12, Target:13, Cust_o.:323, Cust_t.:279, Fits! --- Diff. Cost Sum=99.07345625661301
Origin:12, Target:13, Cust_o.:323, Cust_t.:258, Fits! --- Diff. Cost Sum=97.8089947788063
Origin:12, Target:13, Cust_o.:323, Cust_t.:236, Fits! --- Diff. Cost Sum=96.68483377111127
Origin:12, Target:14, Cust_o.:323, Cust_t.:134, Fits! --- Diff. Cost Sum=93.416602007

KeyboardInterrupt: 